In [71]:
import json
import re
from langchain.chat_models  import AzureChatOpenAI
from langchain.schema import SystemMessage, HumanMessage
from backend.src.utils import MySQLAgent

In [42]:
def read_config(path):
    try:
        with open(path, 'r') as file:
            configs = json.load(file)

        return configs
    except FileNotFoundError:
        print(f"The file {path} was not found.")
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {path}.")

configs = read_config(path="./backend/conn/connections.json")
job_configs = configs["CREDITREPORT"]['Crawler_mysql_conn_info']
sql_agent = MySQLAgent(job_configs)

In [50]:
query = """
SELECT judgement_date, querykey, subjectkey, judgement_text01
FROM judgement_result
WHERE business_accounting_no = 83387850 AND judgement_date is not null
"""
df_judgement = sql_agent.read_table(query=query)

In [54]:
df_judgement = df_judgement.drop_duplicates()
df_judgement

,judgement_date,querykey,subjectkey,judgement_text01
0,2001-06-07,本票裁定,偽造文書等,臺灣高等法院高雄分院刑事判決 九十年度上易字第六０六號\n 上 訴 人...
1,2001-06-07,本票債權,偽造文書等,臺灣高等法院高雄分院刑事判決 九十年度上易字第六０六號\n 上 訴 人...
2,2000-12-14,本票債權,偽造文書等,臺灣高雄地方法院刑事判決 八十八年度易字第一一三二號\n 公 訴 人...
3,2000-10-13,貨款給付,執行異議,臺灣臺中地方法院民事判決 八十九年度簡上字第三五三號\n 上 訴 人...
4,2020-10-07,給付貨款,請求給付貨款,最高法院民事裁定 109年度台上字第1064號\n上 訴 人 傳福企業股份...
5,2018-11-07,給付貨款,給付貨款,臺灣高等法院民事判決 106年度上字第1643號\n上 訴 人 傳福企業股份有...
6,2017-11-13,給付貨款,給付貨款,臺灣臺北地方法院民事判決 105年度訴字第2668號\n原 告 三芳化學工業...
7,2010-08-20,給付貨款,給付貨款,臺灣彰化地方法院民事判決 99年度訴字第581號\n原 告 三芳化學工業股份有限公司\n...
8,2010-06-23,給付貨款,給付貨款,臺灣彰化地方法院民事裁定 99年度補字第266號\n原 告 三芳化學工業股...
9,2004-02-12,給付貨款,給付貨款,最高法院民事判決 九十三年度台上字第二二九號\n 上 訴 人...


## Statistic

querykey 

In [63]:
df_judgement.groupby('querykey').size().to_dict()

{'支付命令': 6, '本票債權': 2, '本票裁定': 1, '異議之訴': 3, '給付票款': 3, '給付貨款': 12, '貨款給付': 1}

In [62]:
df_judgement.groupby(['querykey', 'subjectkey']).size().reset_index().to_dict()

{'querykey': {0: '支付命令',
  1: '支付命令',
  2: '支付命令',
  3: '支付命令',
  4: '支付命令',
  5: '本票債權',
  6: '本票裁定',
  7: '異議之訴',
  8: '異議之訴',
  9: '異議之訴',
  10: '給付票款',
  11: '給付票款',
  12: '給付貨款',
  13: '給付貨款',
  14: '給付貨款',
  15: '給付貨款',
  16: '給付貨款',
  17: '貨款給付'},
 'subjectkey': {0: '偽造文書等',
  1: '損害賠償',
  2: '支付命令',
  3: '給付貨款',
  4: '限期起訴',
  5: '偽造文書等',
  6: '偽造文書等',
  7: '偽造文書等',
  8: '執行異議',
  9: '確認工程款債權存在',
  10: '確認工程款債權存在',
  11: '給付貨款',
  12: '執行異議',
  13: '給付貨款',
  14: '給付貨款等',
  15: '給付貨款（損害賠償）',
  16: '請求給付貨款',
  17: '執行異議'},
 0: {0: 1,
  1: 1,
  2: 2,
  3: 1,
  4: 1,
  5: 2,
  6: 1,
  7: 1,
  8: 1,
  9: 1,
  10: 1,
  11: 2,
  12: 1,
  13: 8,
  14: 1,
  15: 1,
  16: 1,
  17: 1}}

## LLM

data clean

In [69]:
df_judgement.iloc[0].judgement_text01

'臺灣高等法院高雄分院刑事判決\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000九十年度上易字第六０六號\n\u3000\u3000上\u3000訴\u3000人\n\u3000\u3000即\u3000被\u3000告\u3000丙○○\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000丁○○\n\u3000\u3000右\u3000一\u3000人\n\u3000\u3000選任辯護人\u3000林樹根律師\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000邱麗妃律師\n\u3000\u3000\u3000\u3000\u3000\u3000\u3000\u3000莊雯琇律師\n\u3000\u3000上\u3000訴\u3000人\n\u3000\u3000即\u3000被\u3000告\u3000甲○○\n\u3000\u3000選任辯護人\u3000康清敬\n右上訴人因偽造文書等案件，不服臺灣高雄地方法院八十八年度易字第一一三二號中\n華民國八十九年十二月十四日第一審判決（起訴案號：臺灣高雄地方法院檢察署八十\n七年度偵字第一四六０四號），提起上訴，本院判決如左：\n    主  文\n上訴駁回。\n    事  實\n一、丁○○曾於民國八十六年間，因侵占案件，經法院判處有期徒刑五月確定，並於\n    八十六年十月二十四日易科罰金執行完畢，仍不知悔改，其係皮孩皮飾有限公司\n    （下稱皮孩公司）及漾格服飾有限公司（下稱漾格公司）之實際負責人，其配偶\n    丙○○於八十五年三月間積欠乙○○○會款及借款共新台幣（下同）五十四萬元\n    ，並於八十六年六月十九日經最高法院判決應給付乙○○○五十四萬元確定在案\n    ，係乙○○○之債務人。乙○○○為強制執行上開債權，乃於八十七年二月十六\n    日就丙○○於第三人即三芳化學工業股份有限公司之退休金一百四十一萬七千八\n    百二十一元向臺灣高雄地方院民事執行處聲請強制執行（臺灣高雄地方院八十六\n    年執字第二六七四四號、八十五年度執全字第二五０七號），嗣丙○○為阻擾乙\n    ○○○於前述民事強制執行程序中受償

In [86]:
def clean_data(data):
    # Remove specific unicode characters
    cleaned_data = re.sub(r'\u3000', '', data)
    
    # Replace multiple spaces with a single space
    cleaned_data = re.sub(r'\s{2,}', ' ', cleaned_data)
    
    # Remove leading and trailing spaces on each line
    cleaned_data = re.sub(r'^\s+|\s+$', '', cleaned_data, flags=re.MULTILINE)
    
    # Remove redundant newlines
    cleaned_data = re.sub(r'\n\s*\n', '\n', cleaned_data)

    cleaned_data = re.sub(r'\n', '', cleaned_data)

    cleaned_data = re.sub(r'\s', '', cleaned_data)
    
    return cleaned_data

def extract_section(data, start_keyword, end_keyword):
    # Use regex to find the text between start_keyword and end_keyword
    pattern = re.compile(f'{start_keyword}(.*?){end_keyword}', re.DOTALL)
    match = pattern.search(data)
    if match:
        return match.group(1).strip()
    else:
        return None

In [85]:
clean_test = clean_data(df_judgement.iloc[2].judgement_text01)
clean_test

'臺灣高雄地方法院刑事判決八十八年度易字第一一三二號公訴人臺灣高雄地方法院檢察署檢察官被告丙○○被告丁○○被告甲○○右一人選任辯護人林樹根律師邱麗妃律師莊雯琇律師右列被告因偽造文書等案件，經檢察官提起公訴（八十七年度偵字第一四六○四號），本院判決如左：主文丙○○、甲○○共同行使明知為不實之事項，而使公務員登載於職務上所掌之公文書，足以生損害於公眾及他人，各處有期徒刑伍月，如易科罰金均以參佰元折算壹日。丁○○共同行使明知為不實之事項，而使公務員登載於職務上所掌之公文書，足以生損害於公眾及他人，累犯，處有期徒刑陸月，如易科罰金以參佰元折算壹日。事實一、丙○○於民國八十五年三月間積欠乙○○○會款及借款共新台幣（下同）五十四萬元，並於八十六年六月十九日經最高法院判決應給付乙○○○五十四萬元確定在案，係乙○○○之債務人。乙○○○為強制執行上開債權，乃於八十七年二月十六日就丙○○於第三人即三芳化學工業股份有限公司之退休金一百四十一萬七千八百二十一元向臺灣高雄地方院民事執行處聲請強制執行（臺灣高雄地方院八十六年執字第二六七四四號），嗣後丙○○為為阻擾乙○○○於前述民事強制執行程序中受償，並為隱匿自己之財產，遂與其夫丁○○（係批孩公司及漾格公司之實際負責人，前曾於八十六年間，因侵占案件，經臺灣高雄地方法院判處有期徒刑五月，而於八十六年十月二十四日易科罰金執行完畢），及友人甲○○共同基於使公務員登載不實及意圖損害債權人乙○○○之債權之概括犯意聯絡及行為分擔，明知丙○○與甲○○並無債權債務關係，竟於八十七年初，於不詳地點，由甲○○填載丙○○為發票人，而由丙○○本人蓋章，簽發發票日分別為八十四年三月十六日、同年五月十六日、同年六月十六日、同年六月二十日、同年三月九日，到期日分別為八十四年四月十六日、同年六月十六日、同年七月十六日、同年七月二十日、同年四月九日，金額分別為一百萬元四張、一百三十萬元（金額共五百三十萬元），票號分別為ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000、ＴＨ0000000之本票五紙，交由甲○○持向臺灣高雄地方法院聲請裁定准許本票強制執行，致使臺灣高雄地方法院承辦是項公務之法官，僅經由形式審查，於八十七年二月二日在其辦公處所內，將丙○○積欠甲○○票據票款五百三十萬元之不實事項，登載於職務上所掌之民事裁定（臺灣高雄地方法院八十七年票字第一

In [87]:
start_keyword = "主文"
end_keyword = "事實"

result = extract_section(clean_test, start_keyword, end_keyword)
result

'丙○○、甲○○共同行使明知為不實之事項，而使公務員登載於職務上所掌之公文書，足以生損害於公眾及他人，各處有期徒刑伍月，如易科罰金均以參佰元折算壹日。丁○○共同行使明知為不實之事項，而使公務員登載於職務上所掌之公文書，足以生損害於公眾及他人，累犯，處有期徒刑陸月，如易科罰金以參佰元折算壹日。'

model

In [64]:
OPENAI_API_BASE = 'https://yfyai-aoai-instance.openai.azure.com/'
OPEN_AI_VERSION = '2023-05-15'
GPT_DEPLOYMENT_NAME = 'gpt-4-32k'
OPENAI_API_KEY = 'b4fa163a80a945c4a74ae80d76902868'
OPENAI_API_TYPE = 'azure'

In [66]:
llm = AzureChatOpenAI(
    azure_endpoint=OPENAI_API_BASE,
    openai_api_version=OPEN_AI_VERSION,
    azure_deployment=GPT_DEPLOYMENT_NAME,
    openai_api_key=OPENAI_API_KEY,
    openai_api_type="azure",
)

In [88]:
# 加上該公司在主文裡面扮演的腳色，或是他受到什麼影響

messages = [
    SystemMessage(content="你是一個專業的摘要統整AI助手，針對我提出的問題使用繁體中文回覆。"),
    HumanMessage(content=f"閱讀以下內容，告訴我摘要是在說什麼:{result}")
]

llm(messages)

AIMessage(content='這段摘要是在說兩個案件的判決內容。丙○○和甲○○因為共同做出使公務員在公文書中登載不實事項的行為，此行為可能對公眾和他人造成損害，因此被各判有期徒刑五個月，或者可以用每日三百元的罰金來折算。丁○○同樣犯下使公務員在公文書中登載不實事項的行為，且屬於累犯，所以被判處有期徒刑六個月，或者可以用每日三百元的罰金來折算。', response_metadata={'token_usage': {'completion_tokens': 230, 'prompt_tokens': 297, 'total_tokens': 527}, 'model_name': 'gpt-4-32k', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dc5f9643-c8b5-4efc-9752-86001447be6f-0')

## Data check

In [23]:
query = """
SELECT *
FROM judgement_result
WHERE judgement_no = '臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定'
ORDER BY autoid
"""
df_test = sql_agent.read_table(query=query)

In [29]:
df_test_2 = df_test.drop(['creation_dt','update_dt', 'autoid'], axis=1)

In [35]:
print(f'去重複前資料筆數:{df_test.shape[0]}')
print(f'去重複後資料筆數:{df_test_2.drop_duplicates().shape[0]}')

去重複前資料筆數:10
去重複後資料筆數:5


In [30]:
df_test_2.drop_duplicates()

,business_accounting_no,companyname,judgement_class,judgement_date,judgement_no,querykey,subjectkey,judgement_text01,judgement_hyperlink,datasource_num_id
0,04226091,台燿科技股份有限公司,判決書,2023-12-07,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,給付貨款,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
1,04226091,台燿科技股份有限公司,判決書,2023-11-16,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,給付貨款,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
2,04226091,台燿科技股份有限公司,判決書,2023-10-26,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,給付貨款,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
3,04226091,台燿科技股份有限公司,判決書,2023-11-16,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,支付命令,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
4,04226091,台燿科技股份有限公司,判決書,2023-10-26,臺灣桃園地方法院 112 年度訴字第 2129 號民事裁定,支付命令,給付貨款,,https://judgment.judicial.gov.tw/FJUD/data.asp...,26
